# 🧑‍🎓 Assignment: Orthogonalization in ML
Заполни **Student Info**, затем выполняй задания ниже. Запускай тесты — они начисляют баллы. В конце запусти `[SUBMIT]`. Баллы суммируются автоматически.

## Student Info

In [ ]:

# === Заполнить обязательно ===
STUDENT = {
    "first_name": "Имя",
    "last_name": "Фамилия",
    "group": "Группа",
    "task_id": "ORTHO-01"
}
assert STUDENT["first_name"] != "Имя", "Заполните first_name"
assert STUDENT["last_name"]  != "Фамилия", "Заполните last_name"
assert STUDENT["group"]      != "Группа", "Заполните group"
print("✔ Student Info OK")


## Task 1 — Реализовать Грама–Шмидта (30 баллов)
Реализуй функцию `my_gram_schmidt(V)` — возвращает ортонормированный базис.

In [ ]:

import numpy as np
SCORE = 0  # накапливаем баллы

def my_gram_schmidt(V: np.ndarray) -> np.ndarray:
    U = []
    for v in V.astype(float):
        for u in U:
            v = v - np.dot(v, u) * u
        n = np.linalg.norm(v)
        if n > 1e-12:
            U.append(v / n)
    return np.array(U)

# [TEST] (автобаллы)
V = np.array([[1, 1, 0], [1, 0, 1], [2, 1, 1]], dtype=float)
U = my_gram_schmidt(V)
# проверка формы и ортонормальности
assert U.shape[0] >= 2, "Ожидаем минимум 2 вектора в базисе"
# проверяем почти ортогональность
ok_orth = True
for i in range(len(U)):
    for j in range(i+1, len(U)):
        if abs(np.dot(U[i], U[j])) > 1e-6:
            ok_orth = False
assert ok_orth, "Векторы не ортогональны"
# проверка норм
ok_norm = np.allclose(np.linalg.norm(U, axis=1), 1, atol=1e-6)
assert ok_norm, "Векторы не нормированы"
SCORE += 30
print("✔ Task 1 passed: +30")


## Task 2 — PCA на Iris (30 баллов)
Вычисли PCA до 2 компонент и верни `explained_variance_ratio_` как `VAR2`. Сделай scatterplot (PC1 vs PC2).

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

iris = load_iris()
X = iris.data
y = iris.target

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
VAR2 = float(pca.explained_variance_ratio_.sum())

# [TEST]
assert 0.6 <= VAR2 <= 1.0, f"Слишком низкая суммарная дисперсия для 2 PC: {VAR2}"
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y)
plt.title("Iris PCA (2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()
SCORE += 30
print("✔ Task 2 passed: +30")


## Task 3 — Регуляризация (Ridge) и кривая ошибки (40 баллов)
Обучи Ridge на первых 2 признаках Iris. Подбери `alpha` из списка и сохрани лучший как `BEST_ALPHA`, минимальную MSE — как `BEST_MSE`. Построй график MSE от alpha.

In [ ]:

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

X2 = X[:, :2]
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)

ALPHAS = [0.001, 0.01, 0.1, 1, 10, 100]
mses = []
for a in ALPHAS:
    m = Ridge(alpha=a)
    m.fit(X_train, y_train)
    pred = m.predict(X_test)
    mses.append(mean_squared_error(y_test, pred))

BEST_MSE = float(min(mses))
BEST_ALPHA = float(ALPHAS[int(np.argmin(mses))])

# [TEST]
assert BEST_MSE < 1.5, f"Подозрительно большая MSE: {BEST_MSE}"
assert BEST_ALPHA in ALPHAS, "BEST_ALPHA должен быть из заданного списка"
import matplotlib.pyplot as plt
plt.plot(ALPHAS, mses, marker="o")
plt.xscale("log")
plt.xlabel("alpha")
plt.ylabel("MSE")
plt.title("Ridge: error vs alpha")
plt.show()
SCORE += 40
print("✔ Task 3 passed: +40")


## [SUBMIT] — сформировать результат
Запусти эту ячейку в конце. Она создаст файл `result.json`, который прочитает автопроверка.

In [ ]:

import json, time, os

def write_result(student: dict, score: int, extra: dict=None):
    payload = {
        "student": student,
        "score": int(score),
        "timestamp": int(time.time()),
        "extra": extra or {}
    }
    with open("result.json", "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    print("✔ result.json written")

write_result(STUDENT, SCORE, extra={"VAR2": VAR2, "BEST_ALPHA": BEST_ALPHA, "BEST_MSE": BEST_MSE})
print(f"TOTAL SCORE = {SCORE} / 100")
